In [1]:
# 크롤링 작업을 위한 라이브러리 임포트
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests
import pandas as pd
import json
import time   # 코드 진행 지연을 위한 time 임포트
from selenium.webdriver.common.by import By   # 2022년 7월 이후 selenium 업데이트로 인한 xpath 추적 임포트
from selenium.webdriver.common.keys import Keys

#########################################################
####################  API  입력  부분  ####################
#########################################################

##### 카카오 주소를 좌표(위도, 경도)로 변환 api
## 개인 api key를 할당 받아 적용
api_key = '054a38bc8bc9ee2f8fadaaad44e27f97'
def addr_to_lat_lon(addr):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    
#    print(num, result['documents'][0]['address'])
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])


##### 카카오맵 minimap으로 생성
## No module named 'folium' 라는 에러가 뜬다면
## anaconda cmd 창에서 "pip install folium" 으로 설치
import folium
from folium.plugins import MiniMap

def make_map(df, region):
    # 해당 지역 시/도청 좌표 Search
    yp = addr_to_lat_lon(region)[0]
    xp = addr_to_lat_lon(region)[1]
    
    # 지도 생성하기
    m = folium.Map(location=[xp,yp],   # 뽑은 좌표
                   zoom_start=12)
    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커 추가하기
    for i in range(len(df)):
        print(df['위도'][i],df['경도'][i]])
        folium.Marker(location=[df['위도'][i],df['경도'][i]],
                  popup=df['점수'][i],
                  tooltip=[df['식당명'][i], "\n점수", df['점수'][i]]
                  ).add_to(m)
    return m

In [3]:
region = str(input("검색을 원하는 지역의 \'시, 도\'를 입력해 주세요 : "))
page  = int(input("검색 범위(페이지)를 입력해주세요. : "))
starpoint = input("최대점수를 입력해 주세요(4.5 만점) : ")

driver = webdriver.Chrome("chromedriver")
time.sleep(1)
driver.get('https://map.kakao.com/')
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys('%s 식당\n' % region)
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="info.search.place.sort"]/li[1]/a').send_keys(Keys.ENTER)
time.sleep(2)

n_list = []
s_list = []
a_list = []
px_list = []
py_list = []
ka_list = {}
ka_dict = {}



## 페이지 5보다 작을 때
if page < 5:
    for j in range(1,page+1):
        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')        # 가게명
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")                           # 점수
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")                        # 주소
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            try:
                px_list.append(addr_to_lat_lon(address)[0])
                py_list.append(addr_to_lat_lon(address)[1])
            except:
                pass
            a_list.append(address)
            
        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % (j+1)).send_keys(Keys.ENTER)        # 페이지 넘기기
        time.sleep(2)
        


## 각 묶음 페이지의 1쪽 해당
for j in range(page//5):
    source = driver.page_source
    parsed_source = bs(source, 'html.parser')

    names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')              
    names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
    span_names = names_span_list[1:31:2]
    for i in range(len(span_names)):
        names = span_names[i].text
        n_list.append(names)

    
    div_scores = parsed_source.find_all('em', class_ = "num")
    del div_scores[-1]
    for i in range(len(div_scores)):
        scores = div_scores[i].text
        s_list.append(scores)
    
    
    div_address = parsed_source.find_all('div', class_ = "addr")
    for i in range(len(div_address)):
        address = div_address[i].text.replace('\n','').split('(지번)')[0]
        try:
            px_list.append(addr_to_lat_lon(address)[0])
            py_list.append(addr_to_lat_lon(address)[1])
        except:
            pass
        a_list.append(address)
    time.sleep(2)
    
## 각 묶음 페이지의 2, 3, 4, 5쪽 해당
    for i in range(2,7):
        time.sleep(1)
        if i == 6:
            if page%5 == 0:
                continue
            driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            time.sleep(2)
        else:
            driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % i).send_keys(Keys.ENTER)
            time.sleep(2)

        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            try:
                px_list.append(addr_to_lat_lon(address)[0])
                py_list.append(addr_to_lat_lon(address)[1])
            except:
                pass
            a_list.append(address)

        time.sleep(2)

## 입력한 페이지가 속한 묶음 페이지에 해당
if page > 5:
    for k in range(2,page%5+1):

        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % k).send_keys(Keys.ENTER)
        time.sleep(2)

        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            try:
                px_list.append(addr_to_lat_lon(address)[0])
                py_list.append(addr_to_lat_lon(address)[1])
            except:
                pass
            a_list.append(address)

        time.sleep(2)

## 데이터 딕셔너리 형태로 적재
for i in range(len(n_list)):
    print(n_list[i], s_list[i], a_list[i], round(py_list[i], 2), round(px_list[i], 2))
    /
    ka_dict[i] = (n_list[i], s_list[i], a_list[i],py_list[i],px_list[i])

for i in range(len(ka_dict)):
    if ka_dict[i][1] < starpoint:
        del ka_dict[i]
kakao = pd.DataFrame(ka_dict).T
kakao.columns=['식당명', '점수', '주소', '위도', '경도']
print(f"조회된 식당 갯수 : {len(kakao.index)}개")
make_map(df, region)

검색을 원하는 지역의 '시, 도'를 입력해 주세요 : 전주
검색 범위(페이지)를 입력해주세요. : 3
최대점수를 입력해 주세요(4.5 만점) : 3
베테랑 2.8 전북 전주시 완산구 경기전길 135 35.8134363727781 127.151555162851
삼백집 전주본점 3.4 전북 전주시 완산구 전주객사2길 22 35.8179140859948 127.141132845967
고궁 전주본점 3.1 전북 전주시 덕진구 송천중앙로 33 고궁 35.850101719005 127.119418674502
한국집 2.8 전북 전주시 완산구 어진길 119 35.8165280683498 127.14822129333
전주현대옥 남부시장점 4.3 전북 전주시 완산구 풍남문2길 63 2동 74호 35.8124131813762 127.146605047747
전주왱이콩나물국밥전문점 3.7 전북 전주시 완산구 동문길 88 35.8182809141244 127.149180840317
호남각 4.1 전북 전주시 덕진구 시천로 65 35.8650435286986 127.113658128771
PNB풍년제과 전주 본점 3.8 전북 전주시 완산구 팔달로 180 35.8184395467077 127.147198178164
풍년제과 본점 2.8 전북 전주시 완산구 팔달로 145 35.8154410520602 127.147786955501
한국관 본점 3.3 전북 전주시 덕진구 기린대로 425 35.8390837093397 127.129567237916
진미집 본점 3.3 전북 전주시 완산구 노송여울2길 106 1층 35.8268521576547 127.142423673307
가족회관 3.6 전북 전주시 완산구 전라감영5길 17 2층 35.8170716073059 127.145947030017
조점례남문피순대 3.3 전북 전주시 완산구 풍남문2길 39 5동 117~119호 35.8125800574571 127.147152066561
전일갑오 4.3 전북 전주시 완산구 현무2길 16 35.82021

IndexError: list index out of range